<a href="https://colab.research.google.com/github/Sebas20050700/PIPELINE_AERO-METEREOLOGICO/blob/Visual-Crossing/API_VISUAL_CROSSING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **API VISUAL CROSSING**

In [1]:
import requests
import pandas as pd

def consultar_clima_aeropuertos(lista_aeropuertos):
    """
    ETAPA 1: Ingesta Primaria.
    Recibe códigos ICAO (ej: SPJC, SPZO) y genera las coordenadas
    maestras para OpenSky y SENAMHI.
    """
    url = "https://visual-crossing-weather.p.rapidapi.com/forecast"
    api_key = "a7117854a0msh861c287680abd70p1dfc59jsnfffd314081ec"

    headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": "visual-crossing-weather.p.rapidapi.com"
    }

    resultados = []

    for icao in lista_aeropuertos:
        # CORRECCIÓN AQUÍ: Cambiamos "PE" por "Peru" para evitar irnos a Canadá
        location_query = f"{icao}, Peru"

        querystring = {
            "aggregateHours": "1",
            "location": location_query,
            "contentType": "json",
            "unitGroup": "metric",
            "shortColumnNames": "false"
        }

        print(f"🛰️ ETAPA 1: Geolocalizando y consultando clima global para: {location_query}...")

        try:
            response = requests.get(url, headers=headers, params=querystring)
            response.raise_for_status()
            data = response.json()

            # Ubicación detectada por la API
            loc_id = list(data['locations'].keys())[0]
            current = data['locations'][loc_id]['currentConditions']

            # Diccionario de salida: Este es el "contrato" con SENAMHI y OpenSky
            resultados.append({
                "aeropuerto_id": icao,
                "lat": data['locations'][loc_id].get('latitude'), # <--- Coordenada CORRECTA de Perú
                "lon": data['locations'][loc_id].get('longitude'),# <--- Coordenada CORRECTA de Perú
                "temp_celsius": current.get('temp'),
                "viento_kmh": current.get('wspd'),
                "rafagas_kmh": current.get('wgust'),
                "visibilidad_km": current.get('visibility'),
                "condicion_global": current.get('conditions'),
                "timestamp_utc": current.get('datetime')
            })

        except Exception as e:
            print(f"❌ Error procesando {icao}: {e}")

    return pd.DataFrame(resultados)

# Ejemplo de uso

In [2]:
# ==============================================================================
# 1. CARGAMOS LIBRERÍAS Y LA FUNCIÓN DE SENAMHI
# ==============================================================================
import requests
import pandas as pd
import numpy as np

# CORRECCIÓN: Definimos explícitamente lat_objetivo y lon_objetivo
def encontrar_estacion_cercana(lat_objetivo, lon_objetivo, archivo_csv="MAESTRO_ESTACIONES_SENAMHI_GEO.csv"):
    try:
        # CORRECCIÓN: Usamos la variable 'archivo_csv' en lugar del nombre fijo
        df = pd.read_csv(archivo_csv)
    except FileNotFoundError:
        print(f"❌ Error: No encuentro el archivo '{archivo_csv}' aquí.")
        return None

    # Cálculo de distancia
    df['distancia'] = np.sqrt((df['LATITUD'] - lat_objetivo)**2 + (df['LONGITUD'] - lon_objetivo)**2)
    estacion_ganadora = df.sort_values('distancia').iloc[0]

    return {
        "ESTACION": estacion_ganadora['ESTACION'],
        "LATITUD": estacion_ganadora['LATITUD'],
        "LONGITUD": estacion_ganadora['LONGITUD'],
        "DISTANCIA_KM": round(estacion_ganadora['distancia'] * 111, 2)
    }

# ==============================================================================
# 2. DEFINIMOS LA CONSULTA API (CON LA CORRECCIÓN "PERU")
# ==============================================================================
def consultar_clima_aeropuertos(lista_aeropuertos):
    url = "https://visual-crossing-weather.p.rapidapi.com/forecast"
    api_key = "a7117854a0msh861c287680abd70p1dfc59jsnfffd314081ec" # Tu API Key
    headers = {"x-rapidapi-key": api_key, "x-rapidapi-host": "visual-crossing-weather.p.rapidapi.com"}
    resultados = []

    for icao in lista_aeropuertos:
        # --- ¡AQUÍ ESTÁ LA CORRECCIÓN CLAVE! ---
        location_query = f"{icao}, Peru"  # Usamos "Peru" completo, NO "PE"

        querystring = {"aggregateHours": "1", "location": location_query, "contentType": "json", "unitGroup": "metric", "shortColumnNames": "false"}
        print(f"🛰️ Geolocalizando: {location_query}...")

        try:
            response = requests.get(url, headers=headers, params=querystring)
            data = response.json()
            loc_id = list(data['locations'].keys())[0]
            current = data['locations'][loc_id]['currentConditions']

            resultados.append({
                "aeropuerto_id": icao,
                "lat": data['locations'][loc_id].get('latitude'),
                "lon": data['locations'][loc_id].get('longitude'),
                "temp_celsius": current.get('temp'),
                "condicion_global": current.get('conditions')
            })
        except Exception as e:
            print(f"❌ Error con {icao}: {e}")

    return pd.DataFrame(resultados)

# ==============================================================================
# 3. EJECUCIÓN FINAL (AHORA SÍ FUNCIONARÁ)
# ==============================================================================
ruta_aerea = ["SPJC", "SPZO"]

# Paso 1: API
df_maestro = consultar_clima_aeropuertos(ruta_aerea)

print("\n--- 🛰️ DATA CONFIRMADA (PERÚ) ---")
if not df_maestro.empty:
    print(df_maestro[['aeropuerto_id', 'lat', 'lon']])

    # Paso 2: Unión con SENAMHI
    print("\n--- 🔗 VALIDACIÓN CRUZADA ---")
    for index, fila in df_maestro.iterrows():
        print(f"\n✈️ Analizando: {fila['aeropuerto_id']} (Lat: {fila['lat']}, Lon: {fila['lon']})")

        # Ahora Python YA CONOCE esta función
        estacion = encontrar_estacion_cercana(fila['lat'], fila['lon'])

        if estacion:
            print(f"   ✅ Match SENAMHI: {estacion['ESTACION']} a {estacion['DISTANCIA_KM']} km.")
        else:
            print("   ❌ No se pudo cruzar datos (¿Falta el CSV?).")
else:
    print("❌ No se obtuvieron datos de la API.")

🛰️ Geolocalizando: SPJC, Peru...
🛰️ Geolocalizando: SPZO, Peru...

--- 🛰️ DATA CONFIRMADA (PERÚ) ---
  aeropuerto_id      lat      lon
0          SPJC -12.0562 -77.0268
1          SPZO -12.0562 -77.0268

--- 🔗 VALIDACIÓN CRUZADA ---

✈️ Analizando: SPJC (Lat: -12.0562, Lon: -77.0268)
❌ Error: No encuentro el archivo 'MAESTRO_ESTACIONES_SENAMHI_GEO.csv' aquí.
   ❌ No se pudo cruzar datos (¿Falta el CSV?).

✈️ Analizando: SPZO (Lat: -12.0562, Lon: -77.0268)
❌ Error: No encuentro el archivo 'MAESTRO_ESTACIONES_SENAMHI_GEO.csv' aquí.
   ❌ No se pudo cruzar datos (¿Falta el CSV?).


## **Control de calidad de datos**

In [ ]:
# Dimensiones
print("Filas:", df_maestro.shape[0])
print("Columnas:", df_maestro.shape[1])

# Valores nulos
print(df_maestro.isnull().sum())

# Duplicados
print("Duplicados:", df_maestro.duplicated().sum())

Filas: 2
Columnas: 5
aeropuerto_id       0
lat                 0
lon                 0
temp_celsius        2
condicion_global    2
dtype: int64
Duplicados: 0
